In [146]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, ShuffleSplit

In [147]:
players_df = pd.read_csv("players_merged.csv", encoding = "latin1", low_memory = False )

## Cleaning Tranfermarkt.de Data

In [148]:
#deriving birth_month
players_df['birth_month'] = players_df.apply(lambda row: row['Geburtsdatum.'][5:7],axis=1)

In [150]:
#deriving days_on_team
players_df['current_date'] = datetime.now()
players_df.loc[players_df['Im.Team.seit.'] == '-', 'Im.Team.seit.'] = '01.01.1900'
players_df['Im.Team.seit.'] = players_df['Im.Team.seit.'].fillna('01.01.1900')
players_df['days_on_team'] = players_df.apply(lambda row: row['current_date'] - datetime.strptime(row['Im.Team.seit.'],'%d.%m.%Y') , axis = 1)
players_df['days_on_team'] = players_df['days_on_team'].astype('timedelta64[D]')
players_df.loc[players_df['days_on_team'] > 30000, 'days_on_team'] = None

In [151]:
#deriving days_contr_end
players_df.loc[players_df['Vertrag.bis.'] == '-', 'Vertrag.bis.'] = '01.01.1900'
players_df['Vertrag.bis.'] = players_df['Vertrag.bis.'].fillna('01.01.1900')
players_df['days_contr_end'] = players_df.apply(lambda row: datetime.strptime(row['Vertrag.bis.'],'%d.%m.%Y') - row['current_date'], axis = 1)
players_df['days_contr_end'] = players_df['days_contr_end'].astype('timedelta64[D]')
players_df.loc[players_df['days_contr_end'] < -10, 'days_contr_end'] = None

In [153]:
#deriving days_since_contr
players_df['Letzte.Vertragsverlängerung.'] =  players_df['Letzte.Vertragsverlängerung.'].fillna('01.01.1900')
players_df['days_since_contr'] = players_df.apply(lambda row: row['current_date'] - datetime.strptime(row['Letzte.Vertragsverlängerung.'],'%d.%m.%Y') , axis = 1)
players_df['days_since_contr'] = players_df['days_since_contr'].astype('timedelta64[D]')
players_df.loc[players_df['days_since_contr'] < -10, 'days_since_contr'] = None

In [154]:
#deriving player_agent bool
players_df['Spielerberater.'] =  players_df['Spielerberater.'].fillna(False)
players_df.loc[players_df['Spielerberater.'] != False, 'Spielerberater.'] = True

In [155]:
#changing integer columns 
players_df['Größe.'] = players_df['Größe.'].str.replace("m", "")
players_df['Größe.'] = players_df['Größe.'].str.replace(",", ".")
players_df['Größe.'] = players_df['Größe.'].astype(float)

In [156]:
#moving TM information to beginning of df
cols = list(players_df)
cols_removed = list(players_df)
move_cols = ["Schuhgröße.", 'X2.Verein.', 'Vertragsoption.', 'Ausgeliehen.von.', 'name_length', 'birth_month', 
 'current_date', 'days_on_team', 'days_contr_end', 'days_since_contr']

for element in cols:
    if (element in move_cols):
        cols_removed.remove(element)
               
cols = move_cols + cols_removed 
players_df = players_df[cols]

In [157]:
#in performance data replace all na's with 0 
players_df.iloc[:,33:3787] =  players_df.iloc[:,33:3787].fillna(0)

In [158]:
# in performance data replace all str with int 
players_df.iloc[:,33:3787] = players_df.iloc[:,33:3787].replace('-',0)
players_df.iloc[:,33:3787] = players_df.iloc[:,33:3787].replace('\.','', regex = True)
players_df.iloc[:,33:3787] = players_df.iloc[:,33:3787].astype(int)

In [159]:
# changing market value to correct format
players_df['current_mv'] = players_df['current_mv'].replace('-',0)
players_df['current_mv'] = players_df['current_mv'].fillna(0)
players_df['current_mv'] = players_df['current_mv'].replace(',00 Mio.','000000', regex = True)
players_df['current_mv'] = players_df['current_mv'].replace(',','', regex = True)
players_df['current_mv'] = players_df['current_mv'].replace(' Mio.','0000', regex = True)
players_df['current_mv'] = players_df['current_mv'].replace(' Tsd.','000', regex = True)
players_df['current_mv'] = players_df['current_mv'].replace(' .*','', regex = True)
players_df['current_mv'] = players_df['current_mv'].astype(int)
players_df['highest_mv'] = players_df['highest_mv'].replace('-',0)
players_df['highest_mv'] = players_df['highest_mv'].fillna(0)
players_df['highest_mv'] = players_df['highest_mv'].replace('', 0)
players_df['highest_mv'] = players_df['highest_mv'].replace(',00 Mio.','000000', regex = True)
players_df['highest_mv'] = players_df['highest_mv'].replace(',','', regex = True)
players_df['highest_mv'] = players_df['highest_mv'].replace(' Mio.','0000', regex = True)
players_df['highest_mv'] = players_df['highest_mv'].replace(' Tsd.','000', regex = True)
players_df['highest_mv'] = players_df['highest_mv'].replace(' .*','', regex = True)
players_df['highest_mv'] = players_df['highest_mv'].astype(int)

In [160]:
#removing duplicate and unneccessary columns 
drop_cols = ['name_length','current_date','Unnamed: 0','X',
             'Geburtsdatum.','Vollständiger.Name.','Im.Team.seit.',
             'Vertrag.bis.','Letzte.Vertragsverlängerung.', 'Vertragsoption.',
            'Name.im.Heimatland.']

players_df.drop(drop_cols, axis = 1, inplace=True)

## FIFA DATA 

In [161]:
y = players_df['current_mv'].as_matrix()

In [162]:
#removing unneccessary columns
drop_cols = ['ID', 'name', 'full_name', 'club', 'club_logo',
             'special', 'age', 'league', 'height_cm', 'real_face',
             'flag', 'nationality', 'photo', 'eur_value', 'eur_wage',
             'eur_release_clause','real_face', 'flag', 'nationality',
             'photo', 'eur_value', 'eur_wage', 'eur_release_clause','X2.Verein.',
            'highest_mv', 'current_mv', 'player_name', 'X.1']

players_df.drop(drop_cols, axis = 1, inplace=True)

In [163]:
players_df['Social.Media.']= players_df['Social.Media.'].fillna(0)

In [164]:
#create one-hot-encoding for categorical variables 

one_hot = pd.get_dummies(players_df[['Schuhgröße.','body_type','work_rate_att','work_rate_def','preferred_foot','Ausrüster.', 'Schuhmodell.', 'Ausgeliehen.von.','Geburtsort.',
                                    'Nationalität.', 'Position.', 'Fuß.', 'Spielerberater.', 'Aktueller.Verein.',
                                    'gk', '1_on_1_rush_trait', 'acrobatic_clearance_trait', 'argues_with_officials_trait',
                                    'avoids_using_weaker_foot_trait', 'backs_into_player_trait', 'bicycle_kicks_trait',
                                    'cautious_with_crosses_trait', 'chip_shot_trait', 'chipped_penalty_trait', 'comes_for_crosses_trait',
                                    'corner_specialist_trait', 'diver_trait','dives_into_tackles_trait','diving_header_trait','driven_pass_trait',
 'early_crosser_trait',
 "fan's_favourite_trait",
 'fancy_flicks_trait',
 'finesse_shot_trait',
 'flair_trait',
 'flair_passes_trait',
 'gk_flat_kick_trait',
 'gk_long_throw_trait',
 'gk_up_for_corners_trait',
 'giant_throw_in_trait',
 'inflexible_trait',
 'injury_free_trait',
 'injury_prone_trait',
 'leadership_trait',
 'long_passer_trait',
 'long_shot_taker_trait',
 'long_throw_in_trait',
 'one_club_player_trait',
 'outside_foot_shot_trait',
 'playmaker_trait',
 'power_free_kick_trait',
 'power_header_trait',
 'puncher_trait',
 'rushes_out_of_goal_trait',
 'saves_with_feet_trait',
 'second_wind_trait',
 'selfish_trait',
 'skilled_dribbling_trait',
 'stutter_penalty_trait',
 'swerve_pass_trait',
 'takes_finesse_free_kicks_trait',
 'target_forward_trait',
 'team_player_trait',
 'technical_dribbler_trait',
 'tries_to_beat_defensive_line_trait',
 'poacher_speciality',
 'speedster_speciality',
 'aerial_threat_speciality',
 'dribbler_speciality',
 'playmaker_speciality',
 'engine_speciality',
 'distance_shooter_speciality',
 'crosser_speciality',
 'free_kick_specialist_speciality',
 'tackling_speciality',
 'tactician_speciality',
 'acrobat_speciality',
 'strength_speciality',
 'clinical_finisher_speciality',
 'prefers_rs',
 'prefers_rw',
 'prefers_rf',
 'prefers_ram',
 'prefers_rcm',
 'prefers_rm',
 'prefers_rdm',
 'prefers_rcb',
 'prefers_rb',
 'prefers_rwb',
 'prefers_st',
 'prefers_lw',
 'prefers_cf',
 'prefers_cam',
 'prefers_cm',
 'prefers_lm',
 'prefers_cdm',
 'prefers_cb',
 'prefers_lb',
 'prefers_lwb',
 'prefers_ls',
 'prefers_lf',
 'prefers_lam',
 'prefers_lcm',
 'prefers_ldm',
 'prefers_lcb',
 'prefers_gk']])





players_df = players_df.drop(['Schuhgröße.','body_type','work_rate_att','work_rate_def','preferred_foot','Ausrüster.', 'Schuhmodell.', 'Ausgeliehen.von.','Geburtsort.',
                                    'Nationalität.', 'Position.', 'Fuß.', 'Spielerberater.', 'Aktueller.Verein.',
                                    'gk', '1_on_1_rush_trait', 'acrobatic_clearance_trait', 'argues_with_officials_trait',
                                    'avoids_using_weaker_foot_trait', 'backs_into_player_trait', 'bicycle_kicks_trait',
                                    'cautious_with_crosses_trait', 'chip_shot_trait', 'chipped_penalty_trait', 'comes_for_crosses_trait',
                                    'corner_specialist_trait', 'diver_trait',
 'dives_into_tackles_trait',
 'diving_header_trait',
 'driven_pass_trait',
 'early_crosser_trait',
 "fan's_favourite_trait",
 'fancy_flicks_trait',
 'finesse_shot_trait',
 'flair_trait',
 'flair_passes_trait',
 'gk_flat_kick_trait',
 'gk_long_throw_trait',
 'gk_up_for_corners_trait',
 'giant_throw_in_trait',
 'inflexible_trait',
 'injury_free_trait',
 'injury_prone_trait',
 'leadership_trait',
 'long_passer_trait',
 'long_shot_taker_trait',
 'long_throw_in_trait',
 'one_club_player_trait',
 'outside_foot_shot_trait',
 'playmaker_trait',
 'power_free_kick_trait',
 'power_header_trait',
 'puncher_trait',
 'rushes_out_of_goal_trait',
 'saves_with_feet_trait',
 'second_wind_trait',
 'selfish_trait',
 'skilled_dribbling_trait',
 'stutter_penalty_trait',
 'swerve_pass_trait',
 'takes_finesse_free_kicks_trait',
 'target_forward_trait',
 'team_player_trait',
 'technical_dribbler_trait',
 'tries_to_beat_defensive_line_trait',
 'poacher_speciality',
 'speedster_speciality',
 'aerial_threat_speciality',
 'dribbler_speciality',
 'playmaker_speciality',
 'engine_speciality',
 'distance_shooter_speciality',
 'crosser_speciality',
 'free_kick_specialist_speciality',
 'tackling_speciality',
 'tactician_speciality',
 'acrobat_speciality',
 'strength_speciality',
 'clinical_finisher_speciality',
 'prefers_rs',
 'prefers_rw',
 'prefers_rf',
 'prefers_ram',
 'prefers_rcm',
 'prefers_rm',
 'prefers_rdm',
 'prefers_rcb',
 'prefers_rb',
 'prefers_rwb',
 'prefers_st',
 'prefers_lw',
 'prefers_cf',
 'prefers_cam',
 'prefers_cm',
 'prefers_lm',
 'prefers_cdm',
 'prefers_cb',
 'prefers_lb',
 'prefers_lwb',
 'prefers_ls',
 'prefers_lf',
 'prefers_lam',
 'prefers_lcm',
 'prefers_ldm',
 'prefers_lcb',
 'prefers_gk'], axis = 1 )
players_df = players_df.join(one_hot)

In [183]:
X = players_df
X = X.fillna(0)

## Calibrating Random Forest Regressor

In [184]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2)

In [185]:
n_estimators = np.arange(start=50,stop=200,step=10)
max_features = np.arange(start=1000,stop=2500,step=100)

In [186]:
splitted = ShuffleSplit(n_splits=3,test_size=.15)

In [187]:
rf = RandomForestRegressor()
parameters = {'n_estimators':n_estimators,'max_features':max_features}
tune = GridSearchCV(rf,parameters,cv=splitted)
tune.fit(X_train, y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=None, test_size=0.15, train_size=None),
       error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150, 160, 170,
       180, 190]), 'max_features': array([1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000,
       2100, 2200, 2300, 2400])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [188]:
tune.best_params_

{'max_features': 2000, 'n_estimators': 50}

In [189]:
tune.best_score_

0.74625733281925533

In [195]:
X.head()

,birth_month,days_on_team,days_contr_end,days_since_contr,Alter.,Größe.,Social.Media.,Premier.League,Premier.League.1,Premier.League.2,...,Aktueller.Verein._Vitória Setúbal FC,Aktueller.Verein._Waasland-Beveren,Aktueller.Verein._West Ham United,Aktueller.Verein._Willem II Tilburg,Aktueller.Verein._Wisla Plock,Aktueller.Verein._Wolverhampton Wanderers,Aktueller.Verein._Yeni Malatyaspor,Aktueller.Verein._ZSKA Moskau,Aktueller.Verein._Zaglebie Lubin,Aktueller.Verein._Zenit St. Petersburg
0,12,1516.0,1037.0,783.0,28,1.70,0.0,137,137,5,...,0,0,1,0,0,0,0,0,0,0
1,04,216.0,672.0,43337.0,31,1.65,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10,2248.0,1402.0,80.0,30,1.78,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,09,422.0,672.0,43337.0,27,1.79,0.0,36,36,4,...,0,0,0,0,0,0,0,0,0,0
4,06,576.0,1037.0,43337.0,24,1.89,0.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [193]:
list(X)

['birth_month',
 'days_on_team',
 'days_contr_end',
 'days_since_contr',
 'Alter.',
 'Größe.',
 'Social.Media.',
 'Premier.League',
 'Premier.League.1',
 'Premier.League.2',
 'Premier.League.3',
 'Premier.League.4',
 'Premier.League.5',
 'Championship',
 'Championship.1',
 'Championship.2',
 'Championship.3',
 'Championship.4',
 'Championship.5',
 'FA.Cup',
 'FA.Cup.1',
 'FA.Cup.2',
 'FA.Cup.3',
 'FA.Cup.4',
 'FA.Cup.5',
 'EFL.Cup',
 'EFL.Cup.1',
 'EFL.Cup.2',
 'EFL.Cup.3',
 'EFL.Cup.4',
 'EFL.Cup.5',
 'Champions.League',
 'Champions.League.1',
 'Champions.League.2',
 'Champions.League.3',
 'Champions.League.4',
 'Champions.League.5',
 'U21.PL.Qualifikationsgr..1',
 'U21.PL.Qualifikationsgr..1.1',
 'U21.PL.Qualifikationsgr..1.2',
 'U21.PL.Qualifikationsgr..1.3',
 'U21.PL.Qualifikationsgr..1.4',
 'U21.PL.Qualifikationsgr..1.5',
 'Europa.League',
 'Europa.League.1',
 'Europa.League.2',
 'Europa.League.3',
 'Europa.League.4',
 'Europa.League.5',
 'CL.Quali',
 'CL.Quali.1',
 'CL.Quali.2',


## Calibrating Gradient Boosting Regressor

In [206]:
n_estimators = np.arange(start=100,stop=220,step=20)
max_features = np.arange(start=1,stop=3000,step=500)

In [ ]:
gbr = GradientBoostingRegressor()
parameters = {'max_features':max_features,'n_estimators':n_estimators,'max_depth':[2,3,4],'learning_rate':[0.1,0.2,0.3,0.4]}
tune_gbr = GridSearchCV(gbr,parameters,cv=3)
tune_gbr.fit(X_train, y_train)

In [ ]:
tune_gbr.best_params_

In [ ]:
tune_gbr.best_score_

In [201]:
for row in range(0,3272): # df is the DataFrame
         for col in list(players_df):
             if players_df.get_value(row,col) == '':
                    print(row, col)
                    break

C:\Users\Akleefel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
